In [25]:
import pandas as pd


def load_dataset(dataname):
    train = pd.read_csv('{}/train.tsv'.format(dataname),
                        sep='[\t\f\r\v]', names=['query1', 'query2', 'label'])

    valid = pd.read_csv('{}/dev.tsv'.format(dataname),
                        sep='\t', names=['query1', 'query2', 'label'])

    test = pd.read_csv('{}/test.tsv'.format(dataname),
                       sep='\t', names=['query1', 'query2', 'label'])

    return train, valid, test


name = 'lcqmc'
train, valid, test = load_dataset(name)

C:\Users\99480\AppData\Local\Temp\ipykernel_10128\2363752787.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train = pd.read_csv('{}/train.tsv'.format(dataname),


In [26]:
import jieba

for index, dataset in enumerate([train, valid, test]):
    if index != 2:
        dataset.dropna(axis=0, subset='label', inplace=True)
    dataset['cut_q1'] = dataset.apply(
        lambda row: jieba.lcut(row['query1']) if row['query1'] is not None else [], axis=1)
    dataset['cut_q2'] = dataset.apply(
        lambda row: jieba.lcut(row['query2']) if row['query2'] is not None else [], axis=1)

In [27]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import numpy as np

"""
LineSentence(inp)：格式简单：一句话=一行; 单词已经过预处理并被空格分隔。,inp 为数据路径
    size：是每个词的向量维度；
    window：是词向量训练时的上下文扫描窗口大小，窗口为5就是考虑前5个词和后5个词；
    min-count：设置最低频率，默认是5，如果一个词语在文档中出现的次数小于5，那么就会丢弃；
    workers：是训练的进程数（需要更精准的解释，请指正），默认是当前运行机器的处理器核数。这些参数先记住就可以了。
    sg ({0, 1}, optional) – 模型的训练算法: 1: skip-gram; 0: CBOW
    alpha (float, optional) – 初始学习率
    iter (int, optional) – 迭代次数，默认为5
"""
cut_dataset = np.concatenate((train["cut_q1"], train["cut_q2"]))  # 必须是切完后的列表，而不是拼成一个字符串
output = name + ".model"
model = Word2Vec(cut_dataset, sg=1, vector_size=400, min_count=2, window=5)
model.save(output)

In [28]:
output2 = "word2vec_" + name + ".vector"
model.save(output2)

In [29]:
word2vec_model = Word2Vec.load(output2)
k = word2vec_model.wv.index_to_key
sum([len(i) >= 2 for i in k])

32421